## XGBoost

In [1]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
import os
from pathlib import Path

In [2]:
os.getcwd()

'c:\\Users\\Yashar\\End-to-End-Employee-Classification-with-MLOPs\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Yashar\\End-to-End-Employee-Classification-with-MLOPs'

In [10]:
path = Path('artifacts/data_ingestion/Employee.csv')
print(path)

artifacts\data_ingestion\Employee.csv


In [18]:
data = pd.read_csv(path)
data.head()

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0
3,Masters,2016,Bangalore,3,27,Male,No,5,1
4,Masters,2017,Pune,3,24,Male,Yes,2,1


In [16]:
print(data["LeaveOrNot"].value_counts(normalize=True))

0    0.656136
1    0.343864
Name: LeaveOrNot, dtype: float64


In [17]:
data.shape

(4653, 9)

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4653 entries, 0 to 4652
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Education                  4653 non-null   object
 1   JoiningYear                4653 non-null   int64 
 2   City                       4653 non-null   object
 3   PaymentTier                4653 non-null   int64 
 4   Age                        4653 non-null   int64 
 5   Gender                     4653 non-null   object
 6   EverBenched                4653 non-null   object
 7   ExperienceInCurrentDomain  4653 non-null   int64 
 8   LeaveOrNot                 4653 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 327.3+ KB


In [19]:
print(data["Education"].value_counts())

Bachelors    3601
Masters       873
PHD           179
Name: Education, dtype: int64


In [20]:
print(data["City"].nunique())

3


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline

categorical_cols = ["Education", "City", "Gender", "EverBenched"]
numerical_cols = ["JoiningYear", "PaymentTier", "Age", "ExperienceInCurrentDomain"]

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OrdinalEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", StandardScaler(), numerical_cols)
    ]
)

In [22]:
X = data.drop("LeaveOrNot", axis=1)
y = data["LeaveOrNot"]

# Define preprocessing
categorical_cols = ["City", "Gender", "EverBenched", "Education"]  # Assuming Education is nominal
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols)
    ]
)

# Define pipeline with SMOTE and XGBoost
model = ImbPipeline([
    ("preprocessor", preprocessor),
    ("smote", SMOTE(random_state=42)),
    ("classifier", XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42))
])

# Define parameter grid for GridSearchCV
param_grid = {
    "classifier__max_depth": [3, 5, 7],
    "classifier__learning_rate": [0.01, 0.1, 0.3],
    "classifier__n_estimators": [100, 200]
}

# Stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Grid search with cross-validation
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=skf,
    scoring="f1",
    n_jobs=1,
    verbose=1
)

# Fit grid search
grid_search.fit(X, y)

# Print results
print(f"Best F1-score: {grid_search.best_score_:.4f}")
print(f"Best parameters: {grid_search.best_params_}")

# Evaluate best model with cross-validation
f1_scores = []
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    grid_search.best_estimator_.fit(X_train, y_train)
    y_pred = grid_search.best_estimator_.predict(X_test)
    f1_scores.append(f1_score(y_test, y_pred))

print(f"Average F1-score (best model): {sum(f1_scores)/len(f1_scores):.4f}")
print(f"F1-score std (best model): {pd.Series(f1_scores).std():.4f}")

Fitting 5 folds for each of 18 candidates, totalling 90 fits


c:\Users\Yashar\anaconda3\envs\mlops2\lib\site-packages\xgboost\training.py:183: UserWarning: [17:33:15] WARNING: D:\bld\xgboost-split_1748292851775\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Yashar\anaconda3\envs\mlops2\lib\site-packages\xgboost\training.py:183: UserWarning: [17:33:15] WARNING: D:\bld\xgboost-split_1748292851775\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Yashar\anaconda3\envs\mlops2\lib\site-packages\xgboost\training.py:183: UserWarning: [17:33:15] WARNING: D:\bld\xgboost-split_1748292851775\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Yashar\anaconda3\envs\mlops2\lib\site-packages\xgboost\training.py:183: UserWarning: [17:33:15] WARNING: D:\bld\xgboost-split_1748292851775\work\src\learner.cc:738: 
Parameters: { "

Best F1-score: 0.5960
Best parameters: {'classifier__learning_rate': 0.01, 'classifier__max_depth': 5, 'classifier__n_estimators': 200}


c:\Users\Yashar\anaconda3\envs\mlops2\lib\site-packages\xgboost\training.py:183: UserWarning: [17:33:25] WARNING: D:\bld\xgboost-split_1748292851775\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Yashar\anaconda3\envs\mlops2\lib\site-packages\xgboost\training.py:183: UserWarning: [17:33:25] WARNING: D:\bld\xgboost-split_1748292851775\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Yashar\anaconda3\envs\mlops2\lib\site-packages\xgboost\training.py:183: UserWarning: [17:33:26] WARNING: D:\bld\xgboost-split_1748292851775\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Average F1-score (best model): 0.5960
F1-score std (best model): 0.0319


c:\Users\Yashar\anaconda3\envs\mlops2\lib\site-packages\xgboost\training.py:183: UserWarning: [17:33:26] WARNING: D:\bld\xgboost-split_1748292851775\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Yashar\anaconda3\envs\mlops2\lib\site-packages\xgboost\training.py:183: UserWarning: [17:33:26] WARNING: D:\bld\xgboost-split_1748292851775\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


## PyCaret

In [5]:
import pycaret
print(pycaret.__version__)

3.3.2


In [11]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [6]:
data = pd.read_csv("artifacts/data_ingestion/Employee.csv")

print(data.info())
print(data['LeaveOrNot'].value_counts(normalize=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4653 entries, 0 to 4652
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Education                  4653 non-null   object
 1   JoiningYear                4653 non-null   int64 
 2   City                       4653 non-null   object
 3   PaymentTier                4653 non-null   int64 
 4   Age                        4653 non-null   int64 
 5   Gender                     4653 non-null   object
 6   EverBenched                4653 non-null   object
 7   ExperienceInCurrentDomain  4653 non-null   int64 
 8   LeaveOrNot                 4653 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 327.3+ KB
None
LeaveOrNot
0    0.656136
1    0.343864
Name: proportion, dtype: float64


In [7]:
from pycaret.classification import *

In [8]:
# SetUp

clf = setup(data = data,
            target='LeaveOrNot',
            session_id=42,
            fold=5,
            categorical_features= ["Education", "City", "Gender", "EverBenched"],
            numeric_features=["JoiningYear", "PaymentTier", "Age", "ExperienceInCurrentDomain"],
            fix_imbalance=True, # enable SMOTE to handle inbalance
            fix_imbalance_method='smote',
            preprocess=True,
            data_split_stratify=True, # to ensure about stratify splitting
            normalize=True,
            normalize_method='zscore',
            verbose=True
            )

,Description,Value
0,Session id,42
1,Target,LeaveOrNot
2,Target type,Binary
3,Original data shape,"(4653, 9)"
4,Transformed data shape,"(5670, 13)"
5,Transformed train set shape,"(4274, 13)"
6,Transformed test set shape,"(1396, 13)"
7,Numeric features,4
8,Categorical features,4
9,Preprocess,True


In [9]:
# Compare Models

best_model = compare_models(
    sort='F1',
    fold=5,
    include=['xgboost','rf','lr','gbc','ada'],
    n_select=1 # Return  the best model
)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8413,0.8664,0.6866,0.8232,0.7485,0.6339,0.6397,0.7960
xgboost,Extreme Gradient Boosting,0.8351,0.8545,0.6812,0.8108,0.7398,0.6205,0.6260,1.1100
rf,Random Forest Classifier,0.8268,0.8461,0.6857,0.7858,0.7314,0.6046,0.6084,0.8900
ada,Ada Boost Classifier,0.7992,0.8199,0.6321,0.7470,0.6840,0.5384,0.5430,0.0860
lr,Logistic Regression,0.6896,0.7412,0.6500,0.5402,0.5896,0.3434,0.3477,0.8140


In [10]:
# Tune the best model
tuned_model = tune_model(
    estimator=best_model,
    fold=5,
    optimize="F1",  # Optimize for F1-score
    n_iter=10,  # Number of iterations for random search
    search_library="scikit-learn",  # Use scikit-learn RandomizedSearchCV
    verbose=True
)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8436,0.8655,0.7009,0.8177,0.7548,0.6409,0.6450
1,0.8650,0.8909,0.7009,0.8820,0.7811,0.6854,0.6948
2,0.8464,0.8908,0.7188,0.8131,0.7630,0.6500,0.6527
3,0.8587,0.8792,0.7098,0.8548,0.7756,0.6738,0.6800
4,0.8525,0.8534,0.6830,0.8596,0.7612,0.6565,0.6656
Mean,0.8532,0.8760,0.7027,0.8455,0.7671,0.6613,0.6676
Std,0.0079,0.0147,0.0118,0.0262,0.0097,0.0161,0.0180


Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [12]:
predictions = predict_model(tuned_model)
print(f"Test F1-score: {f1_score(predictions['LeaveOrNot'], predictions['prediction_label']):.4f}")
print(f"Test Precision: {precision_score(predictions['LeaveOrNot'], predictions['prediction_label']):.4f}")
print(f"Test Recall: {recall_score(predictions['LeaveOrNot'], predictions['prediction_label']):.4f}")

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.8517,0.8728,0.6854,0.8545,0.7607,0.6551,0.6635


Test F1-score: 0.7607
Test Precision: 0.8545
Test Recall: 0.6854


# GBC

In [13]:
data

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0
3,Masters,2016,Bangalore,3,27,Male,No,5,1
4,Masters,2017,Pune,3,24,Male,Yes,2,1
...,...,...,...,...,...,...,...,...,...
4648,Bachelors,2013,Bangalore,3,26,Female,No,4,0
4649,Masters,2013,Pune,2,37,Male,No,2,1
4650,Masters,2018,New Delhi,3,27,Male,No,5,1
4651,Bachelors,2012,Bangalore,3,30,Male,Yes,2,0


## Data Transformation

In [20]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
import joblib

In [23]:
data['Education'].value_counts()

Education
Bachelors    3601
Masters       873
PHD           179
Name: count, dtype: int64

In [16]:
# Feature Engineering

data['Tenure'] = 2025 - data['JoiningYear']
data

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot,Tenure
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0,8
1,Bachelors,2013,Pune,1,28,Female,No,3,1,12
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0,11
3,Masters,2016,Bangalore,3,27,Male,No,5,1,9
4,Masters,2017,Pune,3,24,Male,Yes,2,1,8
...,...,...,...,...,...,...,...,...,...,...
4648,Bachelors,2013,Bangalore,3,26,Female,No,4,0,12
4649,Masters,2013,Pune,2,37,Male,No,2,1,12
4650,Masters,2018,New Delhi,3,27,Male,No,5,1,7
4651,Bachelors,2012,Bangalore,3,30,Male,Yes,2,0,13


In [24]:
# Define features and target
X = data.drop("LeaveOrNot", axis=1)
y = data["LeaveOrNot"]

- Education: Use OrdinalEncoder to preserve the ordinal relationship (Bachelors < Masters < PhD). This reduces dimensionality and aligns with the feature’s nature.

- City: Use OneHotEncoder if cardinality is low (e.g., 3–5 cities). If high (>10), use TargetEncoder to avoid excessive dimensionality.

- Gender: Keep OneHotEncoder (or use LabelEncoder for simplicity, as it’s binary).

- EverBenched: Keep OneHotEncoder (or use LabelEncoder for simplicity, as it’s binary).

In [25]:
# Define preprocessor
categorical_nominal = ["City", "Gender", "EverBenched"]
categorical_ordinal = ["Education"]
numerical_cols = ["JoiningYear", "PaymentTier", "Age", "ExperienceInCurrentDomain", "Tenure"]
preprocessor = ColumnTransformer(
    transformers=[
        ("cat_nominal", OneHotEncoder(handle_unknown="ignore"), categorical_nominal),
        ("cat_ordinal", OrdinalEncoder(categories=[["Bachelors", "Masters", "PHD"]]), categorical_ordinal),
        ("num", "passthrough", numerical_cols)
    ]
)

In [26]:
# Fit preprocessor (to be used in training)
preprocessor.fit(X)

# Save preprocessor
joblib.dump(preprocessor, "research/preprocessor.pkl")

['research/preprocessor.pkl']

# Model Trainer

In [27]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score
import pandas as pd

In [29]:
# Load the preprocessor

preprocessor = joblib.load('research/preprocessor.pkl')

In [30]:
# Define the model
model = ImbPipeline([
    ('preprocessor',preprocessor),
    ('smote',SMOTE(random_state=42)),
    ('classifier',GradientBoostingClassifier(random_state=42))
])

In [31]:
# Define parameter Grid

param_grid = {
    "classifier__n_estimators": [100, 200, 300],
    "classifier__learning_rate": [0.01, 0.1, 0.2],
    "classifier__max_depth": [3, 5, 7]
}

In [32]:
# Stratified k-fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Grid search
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=skf,
    scoring="f1",
    n_jobs=1,  # Avoid PicklingError
    verbose=1
)

# Fit grid search
grid_search.fit(X, y)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('cat_nominal',
                                                                         OneHotEncoder(categories='auto',
                                                                                       drop=None,
                                                                                       dtype=<class 'numpy.float64'>,
                                                                                       feature_name_c...
                                                                   random_state=42,
                                                                   subsample=1.0,
                                                                   tol=0.0001,
                                                                   validation_fraction=0.1,
                                                                   verbose=0,
                                                                   warm_start=False))],
                                transform_input=None, verbose=False),
             n_jobs=1,
             param_grid={'classifier__learning_rate': [0.01, 0.1, 0.2],
                         'classifier__max_depth': [3, 5, 7],
                         'classifier__n_estimators': [100, 200, 300]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=1)

In [33]:
grid_search.best_params_

{'classifier__learning_rate': 0.01,
 'classifier__max_depth': 5,
 'classifier__n_estimators': 300}

In [34]:
# Evaluate best model
f1_scores = []
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    grid_search.best_estimator_.fit(X_train, y_train)
    y_pred = grid_search.best_estimator_.predict(X_test)
    f1_scores.append(f1_score(y_test, y_pred))

# Print results
print(f"Best F1-score (GridSearchCV): {grid_search.best_score_:.4f}")
print(f"Best parameters: {grid_search.best_params_}")
print(f"Average F1-score: {sum(f1_scores)/len(f1_scores):.4f}")
print(f"F1-score std: {pd.Series(f1_scores).std():.4f}")

Best F1-score (GridSearchCV): 0.7694
Best parameters: {'classifier__learning_rate': 0.01, 'classifier__max_depth': 5, 'classifier__n_estimators': 300}
Average F1-score: 0.7694
F1-score std: 0.0148


In [36]:
# Save model
joblib.dump(grid_search.best_estimator_, "research/model_gbc_tuned.pkl")

['research/model_gbc_tuned.pkl']